# Build a CNN model to classify an image

In [ ]:
import matplotlib.pyplot as plt
import keras
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# import cifar10 dataset
from keras.datasets import cifar10

In [ ]:
# shuffle and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
print(x_train[0].shape)
image = x_train[2]
plt.imshow(image)
plt.show()

In [ ]:
import numpy as np
print(np.unique(y_train,return_counts=True))
print(np.unique(y_test,return_counts=True))

In [ ]:
print(x_test[0,0:2,0:2,])
#print(y_train[1,0])
#print(y_train)

In [ ]:
# define number of classes
num_classes = 10
# Convert class vectors to binary class matrices.
y_train2 = keras.utils.to_categorical(y_train, num_classes)
y_test2 = keras.utils.to_categorical(y_test, num_classes)
print(y_train2.shape)
print(y_test2.shape)

In [ ]:
# convert input train and test as float and devide it by 255 to have values between 0 and 1
# each pixel can have values from 1 to 255 based on the colour and intensity
x_train2 = x_train.astype('float32')
x_test2 = x_test.astype('float32')
x_train2 /= 255
x_test2 /= 255
print(x_train2.shape)

In [ ]:
input_dim = x_train2.shape[1:]
print(input_dim)
print(type(input_dim))

In [ ]:
# Initialize the model
model = Sequential()

# Create the model with two 32 convolution filters -> pooling layer -> two 64 conv filters -> pooling layer 
#                                                  -> flattening -> fully conncted layer 

# add first convolution layer with 32 filters of 3 x 3 size
# image shape will be 32 x 32 x 10 as 10 filters are used with 'same' padding
model.add(Conv2D(10, (3, 3), padding='same',input_shape=input_dim))
model.add(Activation('relu'))

# add second convolution layer with 32 filters of 3 x 3 size
# image shape will be 30 x 30 x 5 as 5 filters are used without padding
#model.add(Conv2D(5, (3, 3)))
#model.add(Activation('relu'))

# add pooling layer with 2 x 2 pooling - default stride is same as pooling size
# image shape will be 15 x 15 x 5 as pooling size and stride are 2
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))                       # drops 25% of units randomly for each epoch

# add thrid convolution layer with 15 filters of 3 x 3 size
# image shape will be 15 x 15 x 5 as 15 filters are used with 'same' padding
#model.add(Conv2D(5, (3, 3), padding='same'))
#model.add(Activation('relu'))

# add fourth convolution layer with 64 filters of 3 x 3 size
# image shape will be 14 x 14 x 5 as 5 filters are used without padding
#model.add(Conv2D(5, (2, 2)))
#model.add(Activation('relu'))

# add pooling layer with 2 x 2 pooling
# image shape will be 7 x 7 x 5 as pooling size and stride are 2
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))                       # drops 25% of units randomly for each epoch

# add flattening layer to flattens into single dimensional
# number of features will be = 7 x 7 x 5
model.add(Flatten())                           

# add hidden layer with 50 units with RELU as activation and dropout rate of 50%
model.add(Dense(10))                          
model.add(Activation('relu'))
#model.add(Dropout(0.5))

# add output layers with number of classes in target variable with softmax as activation
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
# compile the model
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
# define the batch size and epochs
batch_size = 100
epochs = 2
# run the model with given data
model.fit(x_train2, y_train2,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test2, y_test2),
              shuffle=True)

In [ ]:
model.summary()
# the input image size is 32 x 32 x 3
# for first layer, 32 filters of 3x3 size applied across depth(3) of input image. 
# Hence, number of parameters will be 32 x 3 x 3 x 3 + 32 (Number of intercepts/filters) = 896
# output size remains 32 x 32 as padding is done. the depth of impage is now 32 due to 32 number of filters
# for second layer, 32 filters of 3x3 size applied across depth(32) of input image. 
# Hence, number of parameters will be 32 x 3 x 3 x 32 + 32 (Number of intercepts/filters) = 9248
# output size changed to 30 x 30 as padding is not done. the depth of impage is now 32 due to 32 number of filters
# after pooling done with 2 x 2 size with striding with same size, the image size got reduced to 15 x 15
# for third layer with 64 filters: 64 x 3 x 3 x 32 + 64 = 18496. 
# output size remains 15 x 15 as padding is done. the depth of impage is now 64 due to 64 number of filters
# for fourth layer with 64 filters: 64 x 3 x 3 x 64 + 64 = 36928. 
# output size remains 13 x 13 as padding is not done. the depth of impage is now 64 due to 64 number of filters
# after pooling done with 2 x 2 size with striding with same size, the image size got reduced to 6 x 6
# after flattening, the number of input nodes = 6 x 6 x 64 = 2304
# as the number of nodes in hidden layer is 512, 
# the no. of parameters between input and hidder layer: 2304 x 512 + 512 = 1180160 (including 512 intercepts)
# as the number of nodes in output layer is 10, 
# the no. of parameters between hidder and output layer: 512 x 10 + 10 = 530 (including 10 intercepts)

In [ ]:
# get weights for any layer
model.layers[0].get_weights()[0]

In [ ]:
# get output of any layer
model.layers[0].output[0]

In [ ]:
# predict on test
y_pred = model.predict(x_test2)
print(y_pred.shape)
print(y_pred[0:5,])

In [ ]:
# convert the Y_pred into single dimension
y_pred2 = np.argmax(y_pred, axis=1)
print(y_pred2.shape)
print(y_pred2[0:5])

In [ ]:
# Evaluating using confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print (confusion_matrix(y_test,y_pred2))
print (classification_report(y_test,y_pred2))